## Data Wrangling Project: WeRateDogs

<p style='text-align: justify;'><b>Project Introduction</b>: The dataset which is going to be wrangled is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. WeRateDogs has over 4 million followers and has received international media coverage.</p>

**Given datasets:**<br>
<br>
**1. twitter-archive-enhanced.csv**: The WeRateDogs Twitter archive. It's downloaded via the following URL:<br>
https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv<br>
<br>
**2. image-predictions.tsv**: i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL:<br>
https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv<br>
<br>
**3. The Twitter API & JSON**: Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.<br>
<br>
**Note that:** My twitter developer account application was not approved without a reason or any additional request. Thus, I imported the following files manually:<br>
<br>
File: twitter_api.py<br>
URL:https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5be5fb4c_twitter-api/twitter-api.py<br>
<br>
File: tweet_json.txt<br>
URL: https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5be5fb7d_tweet-json/tweet-json.txt

#### REQUIRED LIBRARIES

In [2]:
import numpy as np
import pandas as pd
import requests
import os
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### DATA GATHERING

In [3]:
# Twitter Archieve
df_1 = pd.read_csv('twitter-archive-enhanced.csv')
df_1.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [4]:
# Image Predictions
imp_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(imp_url)
response #returns 200 if ran successfully

<Response [200]>

In [5]:
with open('image-predictions.tsv', mode='wb') as file: #wb:write binary
    file.write(response.content)

In [6]:
df_2 = pd.read_csv('image-predictions.tsv', sep='\t' )
df_2.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [7]:
# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv

# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_1.tweet_id.values
len(tweet_ids)

2356

In [8]:
tweets_data = []
tweet_file = open('tweet_json.txt', "r")
for line in tweet_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue   
tweet_file.close()
df_3 = pd.DataFrame()
df_3['tweet_id'] = list(map(lambda tweet: tweet['id'], tweets_data))
df_3['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_data))
df_3['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_data))

In [9]:
df_3.head(2)

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819


### DATA ASSESSMENT

In [10]:
#Dataframe Dimensions
print('df_1 contains {} rows\ndf_2 contains {} rows\ndf_3 contains {} rows\n'.format(df_1.shape[0], df_2.shape[0], df_3.shape[0]))

df_1 contains 2356 rows
df_2 contains 2075 rows
df_3 contains 2354 rows



**1. twitter-archive-enhanced.csv**

In [11]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [12]:
df_1.isnull().head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,False,True,True,False,False,False,True,True,True,False,False,False,False,False,False,False,False
1,False,True,True,False,False,False,True,True,True,False,False,False,False,False,False,False,False


In [13]:
df_1.isnull().tail(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2354,False,True,True,False,False,False,True,True,True,False,False,False,False,False,False,False,False
2355,False,True,True,False,False,False,True,True,True,False,False,False,False,False,False,False,False


In [14]:
msg01 = sum(df_1['in_reply_to_status_id'].isnull())
msg02 = sum(df_1['in_reply_to_user_id'].isnull())
msg03 = sum(df_1['retweeted_status_id'].isnull())
msg04 = sum(df_1['retweeted_status_user_id'].isnull())
msg05 = sum(df_1['retweeted_status_timestamp'].isnull())
#
print('Sum of the null values are given below:')
print('{} for {}'.format(msg01, 'in_reply_to_status_id'))
print('{} for {}'.format(msg02, 'in_reply_to_user_id'))
print('{} for {}'.format(msg03, 'retweeted_status_id'))
print('{} for {}'.format(msg04, 'retweeted_status_user_id'))
print('{} for {}'.format(msg05, 'retweeted_status_timestamp'))

Sum of the null values are given below:
2278 for in_reply_to_status_id
2278 for in_reply_to_user_id
2175 for retweeted_status_id
2175 for retweeted_status_user_id
2175 for retweeted_status_timestamp


In [15]:
# Number of rows in which denominator is not equal to 10!
df_1[df_1['rating_denominator']!=10].shape[0]

23

In [16]:
# Number of rows in which numerator is less than 10!
df_1[df_1['rating_numerator']<10].shape[0]

440

In [17]:
# Statistics for ratings
df_1['rating_numerator'].describe()

count    2356.000000
mean       13.126486
std        45.876648
min         0.000000
25%        10.000000
50%        11.000000
75%        12.000000
max      1776.000000
Name: rating_numerator, dtype: float64

Above statistics shows that<br>
* The standard deviation is too high considering the mean
* The maximum value is not logical (most probably it's a typo)
* Most probably there some outliers in this column

In [18]:
#Let's check the ratings (>=10) and their frequencies
df_1[df_1['rating_numerator']>=10].rating_numerator.value_counts().sort_index()

10      461
11      464
12      558
13      351
14       54
15        2
17        1
20        1
24        1
26        1
27        1
44        1
45        1
50        1
60        1
75        2
80        1
84        1
88        1
99        1
121       1
143       1
144       1
165       1
182       1
204       1
420       2
666       1
960       1
1776      1
Name: rating_numerator, dtype: int64

In [19]:
#Number of rows in which the rating is bigger than 20!
sum(df_1[df_1['rating_numerator']>20].rating_numerator.value_counts())

24

**2. image-predictions.tsv**

In [20]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


**3. tweet-json.txt**

In [21]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id          2354 non-null int64
retweet_count     2354 non-null int64
favorite_count    2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [22]:
df_3.describe() #retweet_count and favorite_count should be non-negaative

,tweet_id,retweet_count,favorite_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,3164.797366,8080.968564
std,6.852812e+16,5284.770364,11814.771334
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,624.500000,1415.000000
50%,7.194596e+17,1473.500000,3603.500000
75%,7.993058e+17,3652.000000,10122.250000
max,8.924206e+17,79515.000000,132810.000000


In [23]:
sum(df_3.retweet_count.isnull())

0

In [24]:
sum(df_3.favorite_count.isnull())

0

#### Duplicates

In [25]:
msg07 = sum(df_1.duplicated())
msg08 = sum(df_2.duplicated())
msg09 = sum(df_3.duplicated())
#
print('Number of duplicated rows are given below:')
print('{} for {}'.format(msg07, 'twitter-archive-enhanced'))
print('{} for {}'.format(msg08, 'image-predictions'))
print('{} for {}'.format(msg09, 'tweet-json'))

Number of duplicated rows are given below:
0 for twitter-archive-enhanced
0 for image-predictions
0 for tweet-json


In [26]:
msg10 = sum(df_1['tweet_id'].duplicated())
print('Number of duplicated tweet id\'s:')
print('{} in {}'.format(msg10, 'twitter-archive-enhanced'))

Number of duplicated tweet id's:
0 in twitter-archive-enhanced


In [27]:
msg11 = sum(df_2['tweet_id'].duplicated())
print('Number of duplicated tweet id\'s:')
print('{} in {}'.format(msg10, 'image-predictions'))

Number of duplicated tweet id's:
0 in image-predictions


In [28]:
msg12 = sum(df_3['tweet_id'].duplicated())
print('Number of duplicated tweet id\'s:')
print('{} in {}'.format(msg12, 'tweet-json'))

Number of duplicated tweet id's:
0 in tweet-json


In [29]:
df_3['tweet_id'].index.duplicated()

array([False, False, False, ..., False, False, False], dtype=bool)

### Data Issues
In this section, several data issues (quality and structural issues) are mentioned.

#### ATTENTION for the REVIEWER:
My previous submission is commented as follows:<br>
"The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs. You can see this on Project Motivation -> Key Points section in the classroom."
However, the aim is NOT to remove the rows that numerator > denominator!<br>
The aim is to remove the numerator that >>10 because they're outliers!
(I assume this understood wrong due to my expression.)
Thus, I kept this issue as it is.
In case of not accepting this issue, there are additional quality issues in the "Additional Checks" section. Please consider those issues as well.
Thanks!


#### Quality Issues
1. Twitter-archieve dataframe contains rows in which the denominator is not 10. These rows should be removed. Why the denominator is 10? Because "they're good dogs Brent" (https://knowyourmeme.com/memes/theyre-good-dogs-brent")<br>
2. The tweet-json dataframe has duplicates!<br>
3. Twitter-archieve-enhanced dataframe contains rows in which the rating numerator is much bigger than 10. These rows should be removed.<br>
4. Number of rows are not matched.
<br>
'Twitter_archive_enhanced' contains 2356 rows.
<br>
'image_predictions' contains 2075 rows.
<br>
'tweet-json' contains 2354 rows.
<br>
5. The type of 'tweet_id' should be object however it's int64 in twitter-archieve-enhanced and image-prediction dataframes.
6. Following columns in the "twitter-archieve-enhanced" dataframe contain NAN values in most. Thus, these columns will not provide information to us. These columns should be removed.<br>
*in_reply_to_status_id         78 non-null float64<br>
*in_reply_to_user_id           78 non-null float64<br>
*retweeted_status_id           181 non-null float64<br>
*retweeted_status_user_id      181 non-null float64<br>
*retweeted_status_timestamp    181 non-null object<br>
7. In twitter-archieve-enhanced dataframe, the timestamp column has redundant numbers '+0000'.
8. In twitter-archieve-enhanced dataframe, the type of timestamp column is not datetime! 

#### Structural Issues (Tidiness Issues)
1. There are 3 dataframes. However, only 1 data frame is needed.<br>
2. Twitter-archieve-enhanced dataframe contains 4 columns: doggo, floofer, pupper, puppo. These columns are categories. Thus, these columns can be united in same column.<br>
3. Image-predictions dataframe contains 3 predictions. Instead of 3 different prediction outcome, only the best can be used.

**Note-1:** In this notebook QI stands for 'Quality Issue' and SI stands for 'Structural Issue'.

**Note-2:** The issues given above are just **initally** detected issues in this project. There might be additional issues which is solved in this project.

### DATA CLEANING

In [30]:
#Creating copies of dataframes in purpose of having spares.
#Later, a final master dataframe will be saved after all cleaning operations.
df_1c = df_1.copy()
df_2c = df_2.copy()
df_3c = df_3.copy()

In [31]:
##FROM PREVIOUS SUBMISSION (DE-ACTIVATED)
#Saving the copies in 'comma separated value' format
#df_1c.to_csv('df_1c.csv', encoding='UTF-8', index=False)
#df_2c.to_csv('df_2c.csv', encoding='UTF-8', index=False)
#df_3c.to_csv('df_3c.csv', encoding='UTF-8', index=False)
##Importing the copies of dataframes
#df_1c = pd.read_csv('df_1c.csv')
#df_2c = pd.read_csv('df_2c.csv')
#df_3c = pd.read_csv('df_3c.csv')

In [32]:
df_1c.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
506,812747805718642688,NaN,NaN,2016-12-24 19:52:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Sammy. At first I was like...,6.800555e+17,4.196984e+09,2015-12-24 16:00:30 +0000,https://twitter.com/dog_rates/status/680055455...,10,10,Sammy,None,None,None,None
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713900603...,99,90,None,None,None,None,None


In [33]:
df_2c.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
488,675517828909424640,https://pbs.twimg.com/media/CV_r3v4VAAALvwg.jpg,1,Scottish_deerhound,0.240591,True,groenendael,0.156916,True,flat-coated_retriever,0.090899,True
151,668643542311546881,https://pbs.twimg.com/media/CUd_wYRWUAAZsKr.jpg,1,common_iguana,0.483972,False,frilled_lizard,0.111377,False,sandbar,0.078983,False


In [34]:
df_3c.sample(2)

,tweet_id,retweet_count,favorite_count
1343,704491224099647488,97,840
20,887705289381826560,5609,30779


#### Define-01:
The type of 'tweet_id' should be object however it's int64 in twitter-archieve-enhanced and image-prediction dataframes (QI-5).

In [35]:
#Code-01
df_1c['tweet_id'] = df_1c['tweet_id'].astype(str)
df_2c['tweet_id'] = df_2c['tweet_id'].astype(str)
df_3c['tweet_id'] = df_3c['tweet_id'].astype(str)

In [36]:
#Test-01
df_1c.info();
print('\n')
df_2c.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(2), o

#### Define-02:
Number of rows are not matched in the dataframes (QI-4) and also There are 3 dataframes. However, only 1 data frame is needed (SI-1). 

In [37]:
#NOTE: This code returns favorite_count & retweet_count NAN!!!
subset = ['tweet_id']
df_12 = df_1c.merge(df_2c, on=subset, how='left')
df_123 = df_12.merge(df_3c, on=subset, how='left')

In [38]:
#Test-02
df_123.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
2246,667878741721415682,NaN,NaN,2015-11-21 01:34:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tedrick. He lives on the edge. Needs s...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667878741...,...,0.200373,False,miniature_pinscher,0.106003,True,schipperke,0.104733,True,127.0,409.0
1173,720340705894408192,NaN,NaN,2016-04-13 19:59:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Derek. He just got balled on. Can't ev...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/720340705...,...,0.320126,False,lawn_mower,0.080808,False,viaduct,0.065321,False,1098.0,3131.0


In [39]:
df_123.shape

(2356, 30)

#### Define-03:
Following columns in the "twitter-archieve-enhanced" dataframe contain NAN values in most. Thus, these columns will not provide information to us. These columns should be removed.<br>
*in_reply_to_status_id         78 non-null float64<br>
*in_reply_to_user_id           78 non-null float64<br>
*retweeted_status_id           181 non-null float64<br>
*retweeted_status_user_id      181 non-null float64<br>
*retweeted_status_timestamp    181 non-null object<br>
<br>(QI-6)

In [40]:
#Code-03 & Test-03
drop_cols = ['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp']
df_123.drop(drop_cols, axis=1, inplace=True)
#
for col in df_123.columns: 
    print(col) 

tweet_id
timestamp
source
text
expanded_urls
rating_numerator
rating_denominator
name
doggo
floofer
pupper
puppo
jpg_url
img_num
p1
p1_conf
p1_dog
p2
p2_conf
p2_dog
p3
p3_conf
p3_dog
retweet_count
favorite_count


#### Define-04:
The timestamp column has redundant numbers '+0000' (QI-7). These should be removed.

In [41]:
#Code-04
df_123.timestamp = df_123.timestamp.str.strip('+0000')

In [42]:
#Test-04
pd.DataFrame(df_123.timestamp).head(2)

,timestamp
0,2017-08-01 16:23:56
1,2017-08-01 00:17:27


#### Define-05:
The type of timestamp column should be 'datetime' (QI-8).

In [43]:
#Code-05
df_123.timestamp = pd.to_datetime(df_123.timestamp)

In [44]:
#Test-05
pd.DataFrame(df_123.timestamp).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2356 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 36.8 KB


In [45]:
pd.DataFrame(df_123.timestamp).head(2)

,timestamp
0,2017-08-01 16:23:56
1,2017-08-01 00:17:27


#### Define-06:
Twitter-archieve-enhanced dataframe contains 4 columns: doggo, floofer, pupper, puppo. These columns are categories. Thus, these columns can be united in same column (SI-2).<br>
And also, it should be considered that there are some rows in which there are multiple stages such as doggo-floofer instead of one!

In [46]:
df_123.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 25 columns):
tweet_id              2356 non-null object
timestamp             2356 non-null datetime64[ns]
source                2356 non-null object
text                  2356 non-null object
expanded_urls         2297 non-null object
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
name                  2356 non-null object
doggo                 2356 non-null object
floofer               2356 non-null object
pupper                2356 non-null object
puppo                 2356 non-null object
jpg_url               2075 non-null object
img_num               2075 non-null float64
p1                    2075 non-null object
p1_conf               2075 non-null float64
p1_dog                2075 non-null object
p2                    2075 non-null object
p2_conf               2075 non-null float64
p2_dog                2075 non-null object
p3                    20

In [47]:
#Code-06
#Creating a column called 'stage' to merge the doggo, floofer, pupper and puppo columns
df_123['stage'] = df_123[['doggo', 'floofer', 'pupper', 'puppo']].max(axis=1)
#These are labels thus they're all need to be converted into categories
df_123.stage = df_123.stage.astype('category')
df_123[df_123['stage']!='None'].stage.sample(20)

1685    pupper
701      puppo
1157    pupper
1152    pupper
1811    pupper
418     pupper
731      doggo
973     pupper
1464    pupper
1429    pupper
919      doggo
793     pupper
1606    pupper
764      doggo
1652    pupper
449      doggo
581     pupper
460     pupper
499      doggo
1617    pupper
Name: stage, dtype: category
Categories (5, object): [None, doggo, floofer, pupper, puppo]

In [48]:
#Now, let's dorp doggo, floofer, pupper and puppo columns!
df_123.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

In [49]:
#Test-06
df_123.info() #there shouldn't be 'doggo', 'floofer', 'pupper', 'puppo' columns!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 22 columns):
tweet_id              2356 non-null object
timestamp             2356 non-null datetime64[ns]
source                2356 non-null object
text                  2356 non-null object
expanded_urls         2297 non-null object
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
name                  2356 non-null object
jpg_url               2075 non-null object
img_num               2075 non-null float64
p1                    2075 non-null object
p1_conf               2075 non-null float64
p1_dog                2075 non-null object
p2                    2075 non-null object
p2_conf               2075 non-null float64
p2_dog                2075 non-null object
p3                    2075 non-null object
p3_conf               2075 non-null float64
p3_dog                2075 non-null object
retweet_count         2354 non-null float64
favorite_count        

#### Define-07:
Twitter-archieve-enhanced dataframe contains rows in which the denominator is not 10. These rows should be removed.

In [50]:
#Code-07
wrong_rd = df_123[df_123['rating_denominator']!=10].index
df_123.drop(wrong_rd, axis=0, inplace=True);

In [51]:
#Test-07
df_123[df_123['rating_denominator']!=10].shape[0] #should return 0

0

#### Define-08:
Twitter-archieve-enhanced dataframe contains rows in which the rating is much bigger than 10. These rows should be removed (QI-3).

In [52]:
#Code-08
excessive_rn = df_123[df_123['rating_numerator']>20].index
df_123.drop(excessive_rn, axis=0, inplace=True);

In [53]:
#Test-08
df_123[df_123['rating_numerator']>20].shape[0] #should return 0

0

#### Define-09:
Image-predictions dataframe contains 3 predictions. Instead of 3 different prediction outcome, only the best can be used (SI-3).<br>
First, let's determine the best predictor.

In [54]:
#Code-09
#First let's calculate number of True predictions for 3 predictors
true_p1 = df_123[df_123['p1_dog']==True].shape[0]
true_p2 = df_123[df_123['p2_dog']==True].shape[0]
true_p3 = df_123[df_123['p3_dog']==True].shape[0]
#
print('Number of true predictions for the predictors are given below:')
print('Predictor 1: {}'.format(true_p1))
print('Predictor 2: {}'.format(true_p2))
print('Predictor 3: {}'.format(true_p3))
#
print('\nAccuracy of predictor 2: {}'.format(true_p2/df_123.shape[0]))

Number of true predictions for the predictors are given below:
Predictor 1: 1514
Predictor 2: 1535
Predictor 3: 1481

Accuracy of predictor 2: 0.6604991394148021


Based on the number of true predictions, there is no significant difference between the predictors.<br>
The best predictor has a prediction accuracy around 66% which is weak.<br>
Thus, the true predictions from the 3 predictors can be merged.

In [55]:
#Creating two empty lists
prediction = []
confidence = []

#Defining a function that will take the true predictions
def predictors(line):
    if line['p2_dog']==True:
        prediction.append(line['p2'])
        confidence.append(line['p2_conf'])
    elif line['p1_dog']==True:
        prediction.append(line['p1'])
        confidence.append(line['p1_conf'])
    elif line['p3_dog']==True:
        prediction.append(line['p3'])
        confidence.append(line['p3_conf'])
    else:
        prediction.append('no_prediction')
        confidence.append(0)

#Applying the function to the master dataframe
df_123.apply(predictors, axis=1)

#Adding lists to the master dataframe
df_123['prediction']=prediction
df_123['confidence']=confidence

df_123.head(2)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,...,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,stage,prediction,confidence
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,...,0.085851,False,banana,0.076110,False,8853.0,39467.0,None,no_prediction,0.000000
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,...,0.090647,True,papillon,0.068957,True,6514.0,33819.0,None,Pekinese,0.090647


In [56]:
#Delete unnecessary columns
df_123.drop(['p1','p1_conf','p1_dog','p2','p2_conf','p2_dog',
             'p3','p3_conf','p3_dog'], axis=1, inplace=True)

In [57]:
#Test-09
df_123.sample(3)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,retweet_count,favorite_count,stage,prediction,confidence
173,858471635011153920,2017-04-30 00:02:42,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She just arrived. Used paworit...,https://twitter.com/dog_rates/status/858471635...,13,10,Sophie,https://pbs.twimg.com/media/C-nnZBdXkAAB-wg.jpg,1.0,5271.0,22640.0,None,Cardigan,0.008723
1834,676121918416756736,2015-12-13 19:30:01,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Here we are witnessing a very excited dog. Cle...,https://vine.co/v/iZXg7VpeDAv,8,10,None,NaN,NaN,1297.0,2335.0,None,no_prediction,0.000000
1424,697990423684476929,2016-02-12 03:47:39,"<a href=""http://twitter.com/download/iphone"" r...",This is Doug. He's a Draconian Jabbawockee. Ra...,https://twitter.com/dog_rates/status/697990423...,11,10,Doug,https://pbs.twimg.com/media/Ca_ClYOW0AAsvpE.jpg,2.0,1451.0,3592.0,None,Cardigan,0.015018


In [58]:
#last check
num_predicted = df_123[df_123['prediction']!='no_prediction'].shape[0]
print('Number of true predictions: {}'.format(num_predicted))
print('Proportion of true predictions: {:0.2f}'.format(num_predicted/df_123.shape[0]))

Number of true predictions: 1731
Proportion of true predictions: 0.74


#### Define-10:
The tweet-json dataframe has duplicates (QI-2). This issue is intentionally left to the end because previous cleaning operations might yield another duplicates or simply might drop all duplicates.
Let's check the master dataframe.

In [59]:
#Code-10
sum(df_123.duplicated())

0

One (or more than one) of the cleaning operations above might be produced these duplicates!<br>
Most probably in 'Define-07' (since we used a function there), we produced these duplicates.<br>
An additional check confirms that we did it!<br>
Now, let's drop these duplicates from the master dataframe.

In [60]:
#Dropping duplicates
df_123.drop_duplicates(subset=['tweet_id'], inplace=True)
df_123.shape[0]

2324

In [61]:
#Test-10
df_1c.shape[0] == df_123.shape[0] #should return true

False

#### Additional Checks

##### Attention for Reviewers:
We can always go back to assessment phase (and even to gathering phase) after completing several cleaning operations.<br>
Thus, it would be useful to check out master data frame again at this point.

In [62]:
df_123.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2324 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id              2324 non-null object
timestamp             2324 non-null datetime64[ns]
source                2324 non-null object
text                  2324 non-null object
expanded_urls         2272 non-null object
rating_numerator      2324 non-null int64
rating_denominator    2324 non-null int64
name                  2324 non-null object
jpg_url               2052 non-null object
img_num               2052 non-null float64
retweet_count         2322 non-null float64
favorite_count        2322 non-null float64
stage                 2324 non-null category
prediction            2324 non-null object
confidence            2324 non-null float64
dtypes: category(1), datetime64[ns](1), float64(4), int64(2), object(7)
memory usage: 274.8+ KB


There are some missing data in 'jpg_url' and 'img_num' columns.<br>
However, this will NOT affect our analysis.
Let's check the master dataframe visually.

In [63]:
df_123.sample(50)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,retweet_count,favorite_count,stage,prediction,confidence
60,880465832366813184,2017-06-29 16:39:47,"<a href=""http://twitter.com/download/iphone"" r...",This is Bella. She had her first beach experie...,https://twitter.com/dog_rates/status/880465832...,12,10,Bella,https://pbs.twimg.com/media/DDgK-J4XUAIEV9W.jpg,1.0,6546.0,29075.0,None,Labrador_retriever,0.026329
636,793500921481273345,2016-11-01 17:12:16,"<a href=""http://twitter.com/download/iphone"" r...",This is Fiona. She's an extremely mediocre cop...,https://twitter.com/dog_rates/status/793500921...,12,10,Fiona,https://pbs.twimg.com/media/CwMU34YWIAAz1nU.jpg,2.0,2786.0,11953.0,None,Labrador_retriever,0.219904
1012,747242308580548608,2016-06-27 01:37:04,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This pupper killed this great w...,https://twitter.com/dog_rates/status/704761120...,13,10,None,NaN,NaN,3257.0,0.0,pupper,no_prediction,0.000000
226,848690551926992896,2017-04-03 00:16:10,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending in animals other than dogs...,https://twitter.com/dog_rates/status/848690551...,12,10,None,https://pbs.twimg.com/media/C8cnjHuXsAAoZQf.jpg,1.0,4826.0,27104.0,None,Newfoundland,0.100571
2161,669564461267722241,2015-11-25 17:13:02,"<a href=""http://twitter.com/download/iphone"" r...",This is a Coriander Baton Rouge named Alfredo....,https://twitter.com/dog_rates/status/669564461...,10,10,a,https://pbs.twimg.com/media/CUrFUvDVAAA9H-F.jpg,1.0,133.0,413.0,None,miniature_poodle,0.259920
1069,740365076218183684,2016-06-08 02:09:24,"<a href=""http://twitter.com/download/iphone"" r...",When the photographer forgets to tell you wher...,https://twitter.com/dog_rates/status/740365076...,10,10,None,https://pbs.twimg.com/media/CkZOGhJWsAAHvPv.jpg,1.0,495.0,2727.0,None,no_prediction,0.000000
1816,676606785097199616,2015-12-15 03:36:42,"<a href=""http://twitter.com/download/iphone"" r...",*screeches for a sec and then faints* 12/10 ht...,https://twitter.com/dog_rates/status/676606785...,12,10,None,https://pbs.twimg.com/media/CWPKSGpWcAQN6mw.jpg,1.0,490.0,2039.0,None,malinois,0.142300
2020,672095186491711488,2015-12-02 16:49:14,"<a href=""http://twitter.com/download/iphone"" r...",This is Tuco. That's the toast that killed his...,https://twitter.com/dog_rates/status/672095186...,9,10,Tuco,https://pbs.twimg.com/media/CVPDAR9XIAAm8QB.jpg,1.0,395.0,1063.0,None,French_bulldog,0.140796
1560,688519176466644993,2016-01-17 00:32:18,"<a href=""http://twitter.com/download/iphone"" r...",This pupper is sprouting a flower out of her h...,https://twitter.com/dog_rates/status/688519176...,12,10,None,https://pbs.twimg.com/media/CY4ciRFUMAAovos.jpg,1.0,825.0,2520.0,pupper,Cardigan,0.121052
639,793256262322548741,2016-11-01 01:00:05,"<a href=""http://twitter.com/download/iphone"" r...",Oh h*ck look at this spookling right here. Fri...,https://twitter.com/dog_rates/status/793256262...,12,10,None,https://pbs.twimg.com/media/CwI2XCvXEAEO8mc.jpg,1.0,9714.0,22350.0,None,Walker_hound,0.060574


**Define:** Some of the names seem odd.<br>
They include only 'the', 'a', 'an'...!

In [64]:
name_a = df_123[df_123['name']=='a'].shape[0]
name_an = df_123[df_123['name']=='an'].shape[0]
name_the = df_123[df_123['name']=='the'].shape[0]
print('There are some odd names as follows with their frequencies')
print('a: {}'.format(name_a))
print('an: {}'.format(name_an))
print('the: {}'.format(name_the))

There are some odd names as follows with their frequencies
a: 54
an: 6
the: 8


Either we broke the initial data or the initial data was already broken.<br>
Let's check the initial data externally (in excel).<br>
<br>
***(After visual check in excel)***<br>
Yes! The data was initially broken.<br>
And there are more odd names: "quite', 'just', 'very', 'such', 'not', 'actually'!<br>
Thus, let's we assign 'None' to these odd namings.<br>

In [65]:
#Code
odd_names = ['a','an','the','quite','just','very','such','not','actually']
#
for name in df_123['name']:
    if name in odd_names:
        df_123.name[df_123['name']==name] = 'None'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [66]:
#Test
df_123[df_123['name']=='the'].shape[0] #should return 0

0

**Define:** There are redundant columns.
Master dataframe contains 15 columns and we won't use the following ones:
1. source
2. expanded_urls
3. rating_denominator (we know all equals 10)
4. img_num

It's a good habit to remove unnecessary columns so as to reduce the size of the data.<br>
(Of course we could use those columns for example to show the image of top-10 rated dogs.)

In [67]:
#Code
df_123.drop(columns=['source', 'expanded_urls', 'rating_denominator', 'img_num'], inplace=True)

In [68]:
#Test
df_123.shape[1] #should return 11!

11

In [69]:
df_123.head(2)

,tweet_id,timestamp,text,rating_numerator,name,jpg_url,retweet_count,favorite_count,stage,prediction,confidence
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,13,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,8853.0,39467.0,None,no_prediction,0.000000
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,13,Tilly,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,6514.0,33819.0,None,Pekinese,0.090647


In [70]:
df_123.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2324 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id            2324 non-null object
timestamp           2324 non-null datetime64[ns]
text                2324 non-null object
rating_numerator    2324 non-null int64
name                2324 non-null object
jpg_url             2052 non-null object
retweet_count       2322 non-null float64
favorite_count      2322 non-null float64
stage               2324 non-null category
prediction          2324 non-null object
confidence          2324 non-null float64
dtypes: category(1), datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 202.2+ KB


**Define:** We need an additional column to fullfill to requirements of the research question 1: "Which dog breeds are popular? Find top-10!"<br>
In order to find the popular dog breeds, we should define the 'popularity' first.
We can directly use ratings as a measure of popularity, however in social media popularity comes from sharing (e.g. retweet).
Thus, we can use both number of retweets and number of favorites to define 'popularity' as follows:

In [71]:
#Code
df_123c['popularity'] = df_123c['favorite_count'] + df_123c['retweet_count']

NameError: name 'df_123c' is not defined

In [ ]:
#Test
sum(df_123c.retweet_count + df_123c.favorite_count != df_123c.popularity) #this should return to 0 ideally!

**Comment:** The reason why we got 2 instead of 0 there are two None values in the columns however there is no need to drop these observations, later these columns will be used separately as well!


**SAVING THE MASTER DATAFRAME after ADDITIONAL CHECKS:**<br>

In [ ]:
#At this stage a clean master data frame is created and saved.
df_123c.to_csv('twitter_archive_master.csv', encoding='UTF-8', index=False)
#Now we can work with a single dataframe in the ANALYZE phase.
df_123c = pd.read_csv('twitter_archive_master.csv')

Since there are issues to address above, an additional data-assessment is needed as some insights might be affected and might change.<br>
It's done externally in excel.<br>
And also let's check the data again by using head(), tail(), info() and describe() methods.

In [ ]:
df_123c.head(2)

In [ ]:
df_123c.tail(2)

In [ ]:
df_123c.info();

In [ ]:
df_123c.describe()

Now, everything seem fine!<br>
Let's start to anaylze the data!

### ANALYZE

**Research questions:**
1. Which dog breeds are popular? Find top-10!<br>
2. Is there a time-trend of rating?<br>
3. Is there a correlation between favorites, ratings and number of retweets? How strong is it?<br>
4. Try to find the proportion of genders of dogs.
5. Find the photo of the most favourited dog!

#### 1. Which dog breeds are popular? Find top-10!

In order to find the popular dog breeds, we should define the 'popularity' first.<br>
We can directly use **ratings** as a measure of popularity, however in social media popularity comes from sharing (e.g. retweet).<br>
Thus, we can use both number of retweets and number of favorites to define 'popularity' as follows:<br>
****popularity = favorite_count + retweet_count****

In [ ]:
#Creating a new column: 'popularity'
df_123c['popularity'] = df_123c['favorite_count'] + df_123c['retweet_count']
df_123c.head(2)

In [ ]:
df_123c_pop = df_123c[['prediction', 'popularity']] 
df_123c_pop.groupby(['prediction']).mean().sort_values('popularity',ascending=False).head(10)

#### 2. Is there a time-trend of rating?<br>

In [ ]:
#Convert the timestamp into datetime format
#df_123c['timestamp'] = pd.to_datetime(df_123c.timestamp)

#Plot the diagram
from matplotlib.pyplot import figure
df_123c.plot(x='timestamp', y ='rating_numerator', style = '.', alpha = .8, figsize=(12,8))
plt.title('Rating plot over Time',fontsize=18)
plt.xlabel('Date',fontsize=14)
plt.ylabel('Rating',fontsize=14);

Above diagram shows that ratings increase over time!

#### 3. Is there a correlation between favorites, ratings and number of retweets? How strong is it?

Intuitively, we expect a pairwise correlation rating and between number of favorites. Because we give high ratings to our favorites!<br>
Let's start with those 2.

In [ ]:
sns.set_style('ticks')
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
cor1 = sns.regplot(df_123c.rating_numerator,df_123c.favorite_count, ax=ax);
sns.despine();
plt.title("Ratings and Number of Favorites",fontsize=18);
plt.xlabel('Rating',fontsize=14);
plt.ylabel('Favorites',fontsize=14);
cor1.set(ylim=(0, None));

**Note that:** The scatter plot given  above shows that there are some tweets in which the rating numerator is less than 10. Because **there are haters in social media**! We did not drop these row so as to detect the number of haters later, however we can basically re-plot the above diagram by limiting the minimum-x as 10!

In [ ]:
sns.set_style('ticks')
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
cor1 = sns.regplot(df_123c.rating_numerator,df_123c.favorite_count, ax=ax);
sns.despine();
plt.title("Ratings and Number of Favorites",fontsize=18);
plt.xlabel('Rating',fontsize=14);
plt.ylabel('Favorites',fontsize=14);
cor1.set(ylim=(0, None));
cor1.set(xlim=(10, None));

It's hard to say whether there is a correlation between ratings and favorites.<br>
Let's calculate the pearson's correlation coefficient.

In [ ]:
#Building a correlation table
cor2 = df_123c[['favorite_count', 'retweet_count']]
corr = cor2.corr()
corr.style.background_gradient()

There is a moderate correlation (~70%) between number of favorites and number of retweets.<br>
Now, let's also consider the retweets.

In [ ]:
#Plotting the correlation matrix with a color scale
f,ax = plt.subplots(figsize=(5, 5))
sns.heatmap(df_123c[['rating_numerator', 'favorite_count', 'retweet_count']].corr(), annot=True, linewidths=.5, fmt= '.1f')
plt.title('Correlation Matrix');

In [ ]:
#Building a correlation table
cor3 = df_123c[['rating_numerator', 'favorite_count', 'retweet_count']]
corr = cor3.corr()
corr.style.background_gradient()

**Conclusion:** There is a moderate correlation between number of favorites and number of retweets.<br>
However, those two have a weak correlation with ratings!

#### 4. Try to find the proportion of genders.

Let's check the text column in the masterframe.

In [ ]:
df_123c.text

As it can be seen above, text contains some words (he,she,He's,boy,girl etc.) that describe the gender of the dog!<br>
Let's try to derive gender information based on those words.

In [ ]:
#Gender descriptive words
male = ['he', 'He', 'he\'s', 'He\'s', 'him', 'his', 'boy']
female = ['she', 'She', 'she\'s', 'She\'s', 'her', 'hers', 'girl']

gender = [] #empty list

for text in df_123c['text']:
    #Words in male
    if any(map(lambda v:v in male, text.split())):
        gender.append('male')
    #Words in female
    elif any(map(lambda v:v in female, text.split())):
        gender.append('female')
    # If group or not specified
    else:
        gender.append(None)
#
df_123c['gender'] = gender
df_123c.head(4)

In [ ]:
#Number of genders
num_gen = df_123c[df_123c['gender'].notnull()]['gender'].value_counts()
num_gen

In [ ]:
#Bar-Plot
sex = ('Male', 'Female')
numgen = [num_gen.male,num_gen.female]

y_pos = np.arange(len(sex))
plt.bar(y_pos, numgen, align='center', alpha=0.5)
plt.xticks(y_pos, sex)
plt.title('Number of Dog Genders')

plt.show()

**Conclusion:** It's clear that number of male dogs are significantly higher than female dogs.

#### 5. Find the most favourite dog photo!

In [ ]:
most_fav = max(df_123c['favorite_count'])
most_fav_jpg = df_123c[df_123c['favorite_count']==most_fav].jpg_url[407]
#
photo = open('most_fav_jpg.jpg','wb')
photo.write(requests.get(most_fav_jpg).content)
photo.close()

![title](most_fav_jpg.jpg)

***Reports for wrangling and analysis-insight are loaded to the NB workspace!***<br>
Thanks!